In [1]:
import numpy as np
import pandas as pd

In [2]:

train=pd.read_csv('train.csv')
from sklearn.utils import shuffle
train = shuffle(train)

In [3]:
train.head(
)

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
1215,1216,Towards a scientific blockchain framework for ...,Publishing reproducible analyses is a long-s...,1,0,0,0,0,0
19912,19913,The Hilbert scheme of 11 points in A^3 is irre...,We prove that the Hilbert scheme of 11 point...,0,0,1,0,0,0
6034,6035,Combining Contrast Invariant L1 Data Fidelitie...,This paper focuses on multi-scale approaches...,1,0,1,0,0,0
11253,11254,Ultra high stiffness and thermal conductivity ...,"Recently, single crystalline carbon nitride ...",0,1,0,0,0,0
5265,5266,Low Rank Magnetic Resonance Fingerprinting,Purpose: Magnetic Resonance Fingerprinting (...,1,1,0,0,0,0


In [4]:
test=pd.read_csv('test.csv')

In [5]:

test.head()

,ID,TITLE,ABSTRACT
0,20973,Closed-form Marginal Likelihood in Gamma-Poiss...,We present novel understandings of the Gamma...
1,20974,Laboratory mid-IR spectra of equilibrated and ...,Meteorites contain minerals from Solar Syste...
2,20975,Case For Static AMSDU Aggregation in WLANs,Frame aggregation is a mechanism by which mu...
3,20976,The $Gaia$-ESO Survey: the inner disk intermed...,Milky Way open clusters are very diverse in ...
4,20977,Witness-Functions versus Interpretation-Functi...,Proving that a cryptographic protocol is cor...


In [6]:
train['TITLE']=train['TITLE'].astype(str)
train['ABSTRACT']=train['ABSTRACT'].astype(str)
test['TITLE']=test['TITLE'].astype(str)
test['ABSTRACT']=test['ABSTRACT'].astype(str)

In [7]:
import re
def tweet_cleaner(text):
    mention=re.compile(r'@[A-Za-z0-9]+')
    return mention.sub(r'',text)


train["TITLE"] = train['TITLE'].apply(lambda x :tweet_cleaner(x))
train["ABSTRACT"] = train['ABSTRACT'].apply(lambda x :tweet_cleaner(x))
train['TITLE']=train['TITLE'].apply(lambda x :tweet_cleaner(x))
train['ABSTRACT']=train['ABSTRACT'].apply(lambda x :tweet_cleaner(x))

In [8]:
train['TITLE']=train['TITLE'].str.lower()
train["ABSTRACT"] = train['ABSTRACT'].str.lower()
test['TITLE']=test['TITLE'].str.lower()
test['ABSTRACT']=test['ABSTRACT'].str.lower()
train['ABSTRACT'] = train['ABSTRACT'].str.replace(r'\b\w\b', '').str.replace(r'\s+', ' ')
test['ABSTRACT'] = test['ABSTRACT'].str.replace(r'\b\w\b', '').str.replace(r'\s+', ' ')

In [9]:
import string
def clean(text):
    result = re.sub(r'\d+', '', text)
    result=result.strip()
    result=result.translate(str.maketrans('', '', string.punctuation))
    return result
train["TITLE"] = train['TITLE'].apply(lambda x :clean(x))
train["ABSTRACT"] = train['ABSTRACT'].apply(lambda x :clean(x))
test["TITLE"] = test['TITLE'].apply(lambda x :clean(x))
test["ABSTRACT"] = test['ABSTRACT'].apply(lambda x :clean(x))

In [10]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer

In [11]:
stop_words = set(stopwords.words('english'))
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
def token(text):
    tokens = word_tokenize(text)
    result = [i for i in tokens if not i in stop_words]
    final=''
    for word in result:
        final=final +' ' +(lemmatizer.lemmatize(word))
    return final
def lemma(text):
    tokens = word_tokenize(text)
    final=''
    for word in tokens:
        final=final +' ' +(lemmatizer.lemmatize(word))
    return final

train["ABSTRACT"] = train['ABSTRACT'].apply(lambda x :token(x))
train["TITLE"] = train['TITLE'].apply(lambda x :lemma(x))
test["TITLE"] = test['TITLE'].apply(lambda x :lemma(x))
test["ABSTRACT"] = test['ABSTRACT'].apply(lambda x :token(x))

In [12]:
test.head()
test1=test.drop(['ID'],axis=1)
test1=test1['TITLE']+' '+test1['ABSTRACT']
for s in test1:
    l = s.split() 
    k = [] 
    for i in l: # If condition is used to store unique string  # in another list 'k'  
        if (s.count(i)>1 and (i not in k)or s.count(i)==1): 
            k.append(i) 
    s=' '.join(k)

In [13]:
X=train.drop(['ID','Computer Science', 'Physics', 'Mathematics',
       'Statistics', 'Quantitative Biology', 'Quantitative Finance'],axis=1)
y=train['Computer Science']

In [14]:
X=X['TITLE']+ ' ' +X['ABSTRACT']
for s in X:
    l = s.split() 
    k = [] 
    for i in l: # If condition is used to store unique string  # in another list 'k'  
        if (s.count(i)>1 and (i not in k)or s.count(i)==1): 
            k.append(i) 
    s=' '.join(k)

In [15]:

from sklearn import tree
from sklearn.ensemble import BaggingClassifier,GradientBoostingClassifier,RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from xgboost.sklearn import XGBClassifier

text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                     ('tfidf', TfidfTransformer(sublinear_tf=True)),
                     ('clf',   RidgeClassifier(alpha=1.4, max_iter=10000,solver='sparse_cg',tol=0.001)), #'clf', RandomForestClassifier(n_estimators=350,verbose=2)
                     ])

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [17]:
text_clf.fit(X_train,y_train)
predicted = text_clf.predict(X_test)

In [18]:
from sklearn import metrics
print(metrics.classification_report(y_test, predicted))

              precision    recall  f1-score   support

           0       0.92      0.87      0.89      2450
           1       0.83      0.89      0.86      1745

    accuracy                           0.88      4195
   macro avg       0.87      0.88      0.88      4195
weighted avg       0.88      0.88      0.88      4195



In [19]:
accuracy = accuracy_score(y_test, predicted)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 87.87%


In [20]:
text_clf.fit(X,y)
test_predit=text_clf.predict(test1)

In [21]:
final=pd.DataFrame()
final['ID']=test['ID']
final['Computer Science']=test_predit
final.head()

,ID,Computer Science
0,20973,0
1,20974,0
2,20975,1
3,20976,0
4,20977,1


In [22]:
X=train.drop(['ID','Computer Science', 'Physics', 'Mathematics',
       'Statistics', 'Quantitative Biology', 'Quantitative Finance'],axis=1)
y=train['Physics']
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                     ('tfidf', TfidfTransformer(sublinear_tf=True)),
                     ('clf',  RidgeClassifier( max_iter=10000,solver='sparse_cg',tol=0.01)), #'clf', RandomForestClassifier(n_estimators=350,verbose=2)
                     ])
X=X['TITLE']+ ' ' +X['ABSTRACT']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
text_clf.fit(X_train,y_train)
predicted = text_clf.predict(X_test)
accuracy = accuracy_score(y_test, predicted)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 93.80%


In [23]:
text_clf.fit(X,y)
test_predit=text_clf.predict(test1)

In [24]:
final['Physics']=test_predit

In [25]:
final.head()

,ID,Computer Science,Physics
0,20973,0,0
1,20974,0,1
2,20975,1,0
3,20976,0,1
4,20977,1,0


In [26]:
X=train.drop(['ID','Computer Science', 'Physics', 'Mathematics',
       'Statistics', 'Quantitative Biology', 'Quantitative Finance'],axis=1)
y=train['Mathematics']
text_clf = Pipeline([('vect',CountVectorizer(ngram_range=(1, 2))),
                     ('tfidf', TfidfTransformer(sublinear_tf=True)),
                     ('clf',RidgeClassifier(alpha=1.0,max_iter=10000,solver='sparse_cg')), #'clf', RandomForestClassifier(n_estimators=350,verbose=2)
                     ])
X=X['TITLE']+ ' ' +X['ABSTRACT']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
text_clf.fit(X_train,y_train)
predicted = text_clf.predict(X_test)
accuracy = accuracy_score(y_test, predicted)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 90.77%


In [27]:
text_clf.fit(X,y)
test_predit=text_clf.predict(test1)

In [28]:
final['Mathematics']=test_predit

In [29]:
final.head()

,ID,Computer Science,Physics,Mathematics
0,20973,0,0,0
1,20974,0,1,0
2,20975,1,0,0
3,20976,0,1,0
4,20977,1,0,0


In [30]:
X=train.drop(['ID','Computer Science', 'Physics', 'Mathematics',
       'Statistics', 'Quantitative Biology', 'Quantitative Finance'],axis=1)
y=train['Statistics']
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                     ('tfidf', TfidfTransformer(sublinear_tf=True)),
                     ('clf',  RidgeClassifier(alpha=1.05,max_iter=10000,solver='sparse_cg')), #'clf', RandomForestClassifier(n_estimators=350,verbose=2)
                     ])
X=X['TITLE']+ ' ' +X['ABSTRACT']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
text_clf.fit(X_train,y_train)
predicted = text_clf.predict(X_test)
accuracy = accuracy_score(y_test, predicted)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 90.15%


In [31]:
text_clf.fit(X,y)
test_predit=text_clf.predict(test1)

In [32]:
final['Statistics']=test_predit
final.head()

,ID,Computer Science,Physics,Mathematics,Statistics
0,20973,0,0,0,1
1,20974,0,1,0,0
2,20975,1,0,0,0
3,20976,0,1,0,0
4,20977,1,0,0,0


In [33]:
X=train.drop(['ID','Computer Science', 'Physics', 'Mathematics',
       'Statistics', 'Quantitative Biology', 'Quantitative Finance'],axis=1)
y=train['Quantitative Biology']
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                     ('tfidf', TfidfTransformer(sublinear_tf=True)),
                     ('clf',  RidgeClassifier(max_iter=10000,solver='sparse_cg')), #'clf', RandomForestClassifier(n_estimators=350,verbose=2)
                     ])
X=X['TITLE']+ ' ' +X['ABSTRACT']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
text_clf.fit(X_train,y_train)
predicted = text_clf.predict(X_test)
accuracy = accuracy_score(y_test, predicted)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 97.54%


In [34]:
text_clf.fit(X,y)
test_predit=text_clf.predict(test1)

In [35]:
final['Quantitative Biology']=test_predit
final.head()

,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology
0,20973,0,0,0,1,0
1,20974,0,1,0,0,0
2,20975,1,0,0,0,0
3,20976,0,1,0,0,0
4,20977,1,0,0,0,0


In [36]:
X=train.drop(['ID','Computer Science', 'Physics', 'Mathematics',
       'Statistics', 'Quantitative Biology', 'Quantitative Finance'],axis=1)
y=train['Quantitative Finance']
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                     ('tfidf', TfidfTransformer(sublinear_tf=True)),
                     ('clf',RidgeClassifier(max_iter=1000,solver='sparse_cg')), #'clf', RandomForestClassifier(n_estimators=350,verbose=2)
                     ])
X=X['TITLE']+ ' ' +X['ABSTRACT']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
text_clf.fit(X_train,y_train)
predicted = text_clf.predict(X_test)
accuracy = accuracy_score(y_test, predicted)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 99.24%


In [37]:
text_clf.fit(X,y)
test_predit=text_clf.predict(test1)

In [38]:
final['Quantitative Finance']=test_predit
final.head()

,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,20973,0,0,0,1,0,0
1,20974,0,1,0,0,0,0
2,20975,1,0,0,0,0,0
3,20976,0,1,0,0,0,0
4,20977,1,0,0,0,0,0


In [39]:
pred=final.reindex(columns=['ID','Computer Science', 'Physics', 'Mathematics',
       'Statistics', 'Quantitative Biology', 'Quantitative Finance'])

In [42]:
pred.to_csv('luck1.csv',index=False)
pred.head()

,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,20973,0,0,0,1,0,0
1,20974,0,1,0,0,0,0
2,20975,1,0,0,0,0,0
3,20976,0,1,0,0,0,0
4,20977,1,0,0,0,0,0


In [41]:
train[train['Quantitative Finance']==1]

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
17572,17573,bidemographic change and current account usin...,paper new contribution aim explore impact bid...,0,0,0,0,0,1
6954,6955,nongaussian stochastic volatility model with ...,work propose model estimating volatility fina...,0,0,0,0,0,1
3394,3395,characterization of catastrophic instability ...,catastrophic event though rare occur occur de...,0,0,0,0,0,1
7136,7137,optimal investmentconsumption problem postret...,study optimal investmentconsumption problem m...,0,0,0,0,0,1
17735,17736,an enhanced initial margin methodology to man...,use cva cover credit risk widely spread limit...,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...
15490,15491,adapting the cva model to lelands framework,consider framework proposed burgard kjaer der...,0,0,0,0,0,1
12048,12049,private information credit risk and graph str...,research investigated potential improving pee...,1,0,0,0,0,1
13990,13991,bayesian meanvariance analysis optimal portfo...,paper solves problem optimal portfolio choice...,0,0,0,0,0,1
11603,11604,a machine learning framework for stock selection,paper demonstrates apply machine learning alg...,0,0,0,1,0,1
